In [ ]:
#imports
import numpy as np
import matplotlib.pyplot as plt
import os
import torch
import argparse
import importlib
import sys;sys.argv=[''] #passes no cmd line arguments
sys.path.append(os.path.join(os.getcwd(),'RAVEN/src/model'))
sys.path.append(os.path.join(os.getcwd(),'RAVEN/src/model/utility/'))

from glob import glob
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils

In [ ]:
#load img
#to load whole dataset will need to write a torch Dataset class

import data; importlib.reload(data)
batch_size=8

datamodule=data.IRAVENDataModule(batch_size=batch_size)
datamodule.setup()
train_dataloader,val_dataloader,test_dataloader=datamodule.train_dataloader(),datamodule.val_dataloader(),datamodule.test_dataloader()

image,target,meta_target,meta_structure=next(iter(train_dataloader))


In [ ]:
def scl_input_transform(input):

    '''
    Take RAVEN standard input tensor (N,16,img_size,img_size) and return 
    input reshaped for SCL ((N,8,c,img_size,img_size),(N,8,c,img_size,img_size))
    '''

    raise NotImplementedError

    return scl_input_transform


In [ ]:
#CNN model
import models.cnn_mlp as cnn_mlp
importlib.reload(cnn_mlp)

model=cnn_mlp.CNN_MLP()
out=model(image)

In [ ]:
from models import scl


SCL=scl.SCL_model
questions,answers=scl_input_transform(image)
logits=SCL(questions,answers)